In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Content Moderation with NLP API

Based on [Using the Natural Language API with Python.ipynb](https://github.com/GoogleCloudPlatform/devrel-demos/blob/main/other/colab/Using%20the%20Natural%20Language%20API%20with%20Python.ipynb)

<table align="left">
  <td style="text-align: center">
    <a href="https://art-analytics.appspot.com/r.html?uaid=G-FHXEFWTT4E&utm_source=aRT-evaluation_rag_use_cases-from_notebook-colab&utm_medium=aRT-clicks&utm_campaign=evaluation_rag_use_cases-from_notebook-colab&destination=evaluation_rag_use_cases-from_notebook-colab&url=https%3A%2F%2Fcolab.sandbox.google.com%2Fgithub%2FGoogleCloudPlatform%2Fapplied-ai-engineering-samples%2Fblob%2Fmain%2Fgenai-on-vertex-ai%2Fvertex_evaluation_services%2Fevaluation-rag-systems%2Fevaluation_rag_use_cases.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://art-analytics.appspot.com/r.html?uaid=G-FHXEFWTT4E&utm_source=aRT-evaluation_rag_use_cases-from_notebook-colab_ent&utm_medium=aRT-clicks&utm_campaign=evaluation_rag_use_cases-from_notebook-colab_ent&destination=evaluation_rag_use_cases-from_notebook-colab_ent&url=https%3A%2F%2Fconsole.cloud.google.com%2Fvertex-ai%2Fcolab%2Fimport%2Fhttps%3A%252F%252Fraw.githubusercontent.com%252FGoogleCloudPlatform%252Fapplied-ai-engineering-samples%252Fmain%252Fgenai-on-vertex-ai%252Fvertex_evaluation_services%252Fevaluation-rag-systems%252Fevaluation_rag_use_cases.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Run in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://art-analytics.appspot.com/r.html?uaid=G-FHXEFWTT4E&utm_source=aRT-evaluation_rag_use_cases-from_notebook-github&utm_medium=aRT-clicks&utm_campaign=evaluation_rag_use_cases-from_notebook-github&destination=evaluation_rag_use_cases-from_notebook-github&url=https%3A%2F%2Fgithub.com%2FGoogleCloudPlatform%2Fapplied-ai-engineering-samples%2Fblob%2Fmain%2Fgenai-on-vertex-ai%2Fvertex_evaluation_services%2Fevaluation-rag-systems%2Fevaluation_rag_use_cases.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://art-analytics.appspot.com/r.html?uaid=G-FHXEFWTT4E&utm_source=aRT-evaluation_rag_use_cases-from_notebook-vai_workbench&utm_medium=aRT-clicks&utm_campaign=evaluation_rag_use_cases-from_notebook-vai_workbench&destination=evaluation_rag_use_cases-from_notebook-vai_workbench&url=https%3A%2F%2Fconsole.cloud.google.com%2Fvertex-ai%2Fworkbench%2Fdeploy-notebook%3Fdownload_url%3Dhttps%3A%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fapplied-ai-engineering-samples%2Fmain%2Fgenai-on-vertex-ai%2Fvertex_evaluation_services%2Fevaluation-rag-systems%2Fevaluation_rag_use_cases.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
</table>

<table align="left">
    <td>Author(s)</td>
    <td>Egon Soares</td>
</table>

![content moderation architecture](images/1.4-content-moderation.png)

The [Natural Language API](https://cloud.google.com/natural-language/docs/) lets you extract information from unstructured text using Google machine learning. In this tutorial, you'll focus on using its Python client library to perform the following:

- Sentiment analysis
- Entity analysis
- Syntax analysis
- Content classification
- Text moderation

This notebook requires a Google Cloud project:

- If needed, [create a new Google Cloud project](https://console.cloud.google.com/cloud-resource-manager).
- Make sure that billing is enabled for your project.
- It uses billable services but not should generate any cost (see the Natural Language API [free monthly thresholds](https://cloud.google.com/natural-language/pricing)).

## Setup

**[Optional]** Set project. If not set, the project will be set automatically according to the environment variable "GOOGLE_CLOUD_PROJECT".

In [ ]:
PROJECT_ID = ""  # @param {type:"string"}

In [ ]:
import os

In [ ]:
if not PROJECT_ID:
    # Get the default cloud project id.
    PROJECT_ID = os.environ.get("GOOGLE_CLOUD_PROJECT", "")
    assert PROJECT_ID, "Provide a google cloud project id."

In [ ]:
import sys

running_in_colab = "google.colab" in sys.modules

if running_in_colab and os.environ.get("VERTEX_PRODUCT", "") != "COLAB_ENTERPRISE":
    from google.colab import auth as colab_auth
    
    colab_auth.authenticate_user()

### Install packages

Install the Natural Language API package and the notebook packages and then restart the session.

In [ ]:
! pip install --quiet google-cloud-language==2.11.0

In [ ]:
! gcloud services enable language.googleapis.com --project $PROJECT_ID

## Using the Python client library

You can use the Natural Language API in Python with the client library `google-cloud-language` and the following import:

In [ ]:
from google.cloud import language_v1 as language
from google.api_core.client_options import ClientOptions

client_options = ClientOptions(quota_project_id=PROJECT_ID)
client = language.LanguageServiceClient(client_options=client_options)

## Text Moderation

Text moderation identifies a wide range of harmful content, including hate speech, bullying, and sexual harassment. It is performed with the `moderate_text` method which returns a `ModerateTextResponse`.

In [ ]:
from IPython.display import HTML, display

def moderate_text(text: str) -> list:
    document = language.Document(
        content=text,
        type_=language.Document.Type.PLAIN_TEXT,
    )
    response = client.moderate_text(document=document)
    columns = ["category", "confidence"]
    categories = response.moderation_categories
    sorted_categories = sorted(categories, key=lambda x: x.confidence, reverse=True)
    scores = [(category.name, category.confidence) for category in sorted_categories]
    return scores

def show_as_table(scores: list, header="Categories"):
    sorted_scores = sorted(scores, key=lambda x: x[1], reverse=True)
    html_table = f"<table><th>{header}</th><th>Scores</th>"
    for row in sorted_scores:
        html_table += f"<tr><td>{row[0]}</td><td>{row[1]:.0%}</td></tr>"
    html_table += "</table>"
    
    display(HTML(html_table))

Analyze a text

In [ ]:
text = "I have to read Ulysses by James Joyce and am a little over halfway through. I hate it. What a pile of garbage!"  # @param {type:"string"}

In [ ]:
# Send a request to the API
moderate_text_scores = moderate_text(text)

# Show the results
show_as_table(moderate_text_scores)

For more information, see the following docs:

- [`ModerateTextResponse`](https://cloud.google.com/python/docs/reference/language/latest/google.cloud.language_v1.types.ModerateTextResponse)
- [Language Support](https://cloud.google.com/natural-language/docs/languages#content_classification)
- [Moderating Text](https://cloud.google.com/natural-language/docs/moderating-text)